In [6]:
from os import listdir
from os.path import isfile, join

# mypath = 'yolov5/Hard-Hat-Sample-1/train/labels'
mypath = 'datasets/coco128/images/train2017'

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [7]:
len(onlyfiles)

129

In [3]:
f'{mypath}/{onlyfiles[5]}'

'yolov5/Hard-Hat-Sample-1/train/labels/000040_jpg.rf.ZWhxaO4BVVAMw9tL1Ggr.txt'

In [4]:
import numpy as np
import pandas as pd
import pickle


hard_hat_dict = {'0':'head','1':'helmet','2':'person'}
coco_dict = {'80':'head','81':'helmet','0':'person'}
coco_dict = dict(map(reversed,coco_dict.items()))


for file_name in onlyfiles:
    lst = []
    with open(f'{mypath}/{file_name}') as small_pf:
        ## text file 엔터 없이 읽기
        tmp_list = []
        for line in small_pf:
            line = line.rstrip("\n")  
            if line == "":
                lst.append(tmp_list)
                tmp_list = []
            else:
                tmp_list.extend(line.split())
        if tmp_list:  # add last one
            lst.append(tmp_list)

        ## 형변환 후 DF로 변경        
        temp_array = np.array(lst)
        temp_df = pd.DataFrame(columns = ['label','x','y','width','height'], data = np.reshape(temp_array, (-1,5)))

        ## 기존 Label과 index 맞추기 위해 변경
        temp_df['label'].replace(hard_hat_dict, inplace =True)
        temp_df['label'].replace(coco_dict, inplace = True)

        ## 형변환
        result_array = np.reshape(np.array(temp_df),(1,-1)).tolist()
        result_list = []
        for i in range(int(len(result_array[0])/5)):
            result_list.append(' '.join(result_array[0][i*5:(i+1)*5]))

    with open(f'datasets/custom_data/labels/train2017/{file_name}', 'w') as f:
        for line in result_list:
            f.write(line)
            f.write('\n')